# 1. 스파크 내부 메타데이터스토어 정보 확인 

## 스파크 세션생성

In [32]:
%%writefile spark_ss.py

from pyspark.sql import SparkSession

class SparkSS :
    def __init__(self, appl_name) :
        self.appl_name = appl_name
    
    def getSpark(self) : 
         self._spark = (SparkSession.builder.appName(self.appl_name)
                .config("spark.driver.host","127.0.0.1") 
                .config("spark.driver.bindAddress","127.0.0.1")
                .getOrCreate())
         
         return self._spark

Overwriting spark_ss.py


## 세션을 모듈로 작성 후 처리 

In [2]:
import spark_ss

In [3]:
spark = spark_ss.SparkSS("subquery_app").getSpark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/13 04:06:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/13 04:06:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/13 04:06:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/09/13 04:06:13 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [4]:
spark

# 2. 파이스파크(SQL):

### SQL 기반: 
- 파이스파크 SQL은 SQL 쿼리를 사용하여 데이터를 처리합니다.
- SQL은 많은 데이터베이스 시스템에서 널리 사용되는 표준 쿼리 언어이므로 SQL 문법에 익숙한 개발자에게는 익숙하고 쉽게 사용할 수 있습니다.

### 선언적: 
- SQL은 선언적 언어로, 무엇을 원하는지만 명시하고 최적화 및 실행은 시스템이 처리합니다.
- 개발자는 데이터 처리 과정을 자세히 제어할 필요가 없습니다.

### 내장 함수 및 집계: 
- SQL은 다양한 내장 함수와 집계 함수를 제공하여 데이터 조작 및 집계를 쉽게 수행할 수 있습니다.

### 최적화: 
- 파이스파크 SQL은 내부적으로 최적화 및 실행 계획을 생성하여 쿼리 성능을 향상시킵니다.

### Structured APIs와 통합: 
- 파이스파크 SQL은 구조화된 API와 연동되어 사용자가 SQL과 메서드 기반 처리를 혼합하여 사용할 수 있습니다.

## 스파크 내부 스키마 확인

In [5]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [6]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



## 데이터프레임 생성 및 템프테이블 처리 

In [7]:
# Create a DataFrame
data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]
df = spark.createDataFrame(data, ["name", "age"])


## 데이터프레임을 임시테이블로 등록 

In [8]:
# Register the DataFrame as a temporary view
df.createOrReplaceTempView("people")

In [9]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |   people|       true|
+---------+---------+-----------+



In [ ]:
##

In [10]:
spark.sql("SELECT AVG(age) AS average_age FROM people").show()

+-----------+
|average_age|
+-----------+
|       30.0|
+-----------+



# 2. 쿼리와 메서드 처리 비교하기 

## 파이스파크 메서드(메서드 처리):

### 메서드 체인: 
- 파이스파크 메서드 처리는 메서드를 연속적으로 체인하여 데이터를 처리합니다. 
- 이 방식은 함수형 프로그래밍 개념을 활용하므로 일련의 데이터 조작 단계를 명확하게 정의할 수 있습니다.

### 명령형: 
- 메서드 처리는 명령형 프로그래밍 방식으로 데이터 처리를 수행합니다. 
- 개발자는 데이터 처리 단계를 직접 명시하고 제어합니다.

### 유연성: 
- 메서드 처리는 SQL보다 유연하며, 복잡한 데이터 처리 작업을 수행하기에 더 적합할 수 있습니다. 
- 사용자 정의 함수를 활용하여 데이터 조작을 더욱 특화시킬 수 있습니다.

### 타입 안정성: 
- 메서드 처리는 타입 안정성을 보장하므로 런타임 오류를 방지할 수 있습니다.


### 함수 내의 내장함수 

In [14]:
from pyspark.sql.functions import avg

### 하나의 칼럼을 조회

In [13]:
df.select('age').show()

+---+
|age|
+---+
| 25|
| 30|
| 35|
+---+



### 하나의 칼럼을 내장함수로 적용 

In [15]:
df.select(avg("Age")).show()

+--------+
|avg(Age)|
+--------+
|    30.0|
+--------+



# 3. 서브쿼리 처리 

## CTE(공통 테이블 표현식) 

- with 구문으로 서브쿼리를 공통으로 표시
- 본 쿼리에서 이를 임시테이블 처럼 사용 

In [11]:
# Use CTE to process a subquery
result = spark.sql("""
    WITH avg_age AS (
        SELECT AVG(age) AS average_age FROM people
    )
    SELECT name FROM people WHERE age > (SELECT average_age FROM avg_age)
""")

# Show the result
result.show()

+-------+
|   name|
+-------+
|Charlie|
+-------+



## select 절에 서브쿼리

In [12]:
# Use a subquery in the SELECT clause
result3 = spark.sql("""
    SELECT name, age, (SELECT AVG(age) FROM people) AS avg_age
    FROM people
""")

# Show the result
result3.show()

+-------+---+-------+
|   name|age|avg_age|
+-------+---+-------+
|  Alice| 25|   30.0|
|    Bob| 30|   30.0|
|Charlie| 35|   30.0|
+-------+---+-------+



## from 절에 서브쿼리 

In [13]:
# Use a subquery in the FROM clause
result2 = spark.sql("""
    SELECT subquery.name, subquery.age
    FROM (SELECT name, age FROM people WHERE age > 30) AS subquery
""")

# Show the result
result2.show()

+-------+---+
|   name|age|
+-------+---+
|Charlie| 35|
+-------+---+



## where 절 서브쿼리 
- 서브쿼리가 하나의 값인지은 비교 등으로 처리
- 서브쿼리의 결과가 여러 개의 값일 경우는 in 연산자로 처리 

In [14]:
spark.sql("SELECT AVG(age) FROM people").show()

+--------+
|avg(age)|
+--------+
|    30.0|
+--------+



In [15]:
# Use a subquery to filter the data
result1 = spark.sql("SELECT name FROM people WHERE age > (SELECT AVG(age) FROM people)")

# Show the result
result1.show()

+-------+
|   name|
+-------+
|Charlie|
+-------+



In [16]:
# Stop the SparkSession
spark.stop()